## Integration der Nachverarbeitung der HYRAS-DATEN

In [ ]:
import os
import pandas as pd

Festlegen der Gebiets-ID

In [ ]:
ID = 1

# create folder camels_de
os.makedirs("../output_data/camels_de", exist_ok=True)

# create subfolder timeseries
os.makedirs("../output_data/camels_de/timeseries", exist_ok=True)

Festlegen der zu verarbeitenden Variablen

In [ ]:
hyras_variables = ["Precipitation", "Humidity", "RadiationGlobal", "TemperatureMean", "TemperatureMin", "TemperatureMax"]
df_mt = pd.DataFrame()

for variable in hyras_variables:
        df_var = pd.read_csv(f"../output_data/{ID}/data/{ID}_{variable}.csv")  # read the data
        df_var.rename(columns={df_var.columns[0]: "date"}, inplace=True)
        df_var["date"] = pd.to_datetime(df_var["date"]).dt.date
        df_var["date"] = pd.to_datetime(df_var["date"])

        if df_mt.empty:
            df_mt = df_var
        else:
            df_mt = pd.merge(df_mt, df_var, on="date", how="right")

df_mt = df_mt.round(2)

# Save the merged dataframe to a csv file
df_mt.to_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{ID}.csv", index=False)

print("HYRAS-Daten erfolgreich verarbeitet und gespeichert!")

Umbennenung der Spalten

In [ ]:

# define the specific data file
data_file = f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{ID}.csv"

# load data
df = pd.read_csv(data_file, index_col=0)

# get columns
columns = df.columns

# rename columns
new_colnames = [col.replace("hurs", "humidity") for col in columns]
new_colnames = [col.replace("rsds", "radiation_global") for col in new_colnames]
new_colnames = [col.replace("tasmin", "temperature_min") for col in new_colnames]
new_colnames = [col.replace("tasmax", "temperature_max") for col in new_colnames]
new_colnames = [col.replace("tas", "temperature_mean") for col in new_colnames]
new_colnames = [col.replace("pr", "precipitation") for col in new_colnames]

# set new column names
df.columns = new_colnames

# further rename specific columns if they exist
df.rename(columns={
    "temperature_mean_mean": "temperature_mean",
    "temperature_min_mean": "temperature_min",
    "temperature_max_mean": "temperature_max"
}, inplace=True)

# confirm changes
print(df.columns)

Es existieren noch Spalten, die im Datensatz nicht enthalten sein sollen, diese können noch aussortiert werden:

In [ ]:

# define the columns to drop
columns_to_drop = ["temperature_mean_stdev", "temperature_mean_min", "temperature_mean_max", "temperature_min_stdev", "temperature_min_min", "temperature_min_max","temperature_max_stdev", "temperature_max_min", "temperature_max_max"]

# drop columns
df.drop(columns=columns_to_drop, inplace=True)

# Dummy Spalte hinzufügen für die Zielvariable
df["discharge_spec_obs"] = 0

# save data
df.to_csv(data_file, index=True)

Nun liegen alle Spalten außer discharge_vol_ob, discharge_spec_obs und water_level_obs, die nicht benötigt werden, da sie ja simuliert werden sollen, in der korrekten Bezeichung vor.